回调函数的一些用法示例如下所示。
- 模型检查点（model checkpointing）：在训练过程中的不同时间点保存模型的当前权重。
- 提前终止（early stopping）：如果验证损失不再改善，则中断训练（当然，同时保存在训练过程中得到的最佳模型）。
- 在训练过程中动态调节某些参数值：比如优化器的学习率。
- 在训练过程中记录训练指标和验证指标，或将模型学到的表示可视化（这些表示也在不断更新）：你熟悉的 Keras 进度条就是一个回调函数！

keras.callbacks 模块包含许多内置的回调函数，下面列出了其中一些，但还有很多没有列出来。
- keras.callbacks.ModelCheckpoint
- keras.callbacks.EarlyStopping
- keras.callbacks.LearningRateScheduler
- keras.callbacks.ReduceLROnPlateau
- keras.callbacks.CSVLogger

### ModelCheckpoint和EarlyStopping回调函数
如果监控的目标指标在设定的轮数内不再改善，可以用 EarlyStopping 回调函数来中断训练。比如，这个回调函数可以在刚开始过拟合的时候就中断训练，从而避免用更少的轮次重新训练模型。

这个回调函数通常与 ModelCheckpoint 结合使用，后者可以在训练过程中持续不断地保存模型（你也可以选择只保存目前的最佳模型，即一轮结束后具有最佳性能的模型）。

In [ ]:
import keras

# 通过fit的callbacks参数将回调函数传入模型中，这个参数接收一个回调函数的列表。
# 你可以传入任意个数的回调函数
callbacks_list = [
    keras.callbacks.EarlyStoppng( # 如果不再改善，就中断训练
        monitor='acc', # 监控模型的验证精度
        patience=1, # 如果精度在多于一轮的时间（即两轮）内不再改善，中断训练
    ),
    keras.callbacks.ModelCheckpoint( # 在每轮过后保存当前权重
        filepath='my_model.h5', # 目标模型文件的保存路径
        monitor='val_loss', # 以下2行的含义是，如果val_loss没有改善，那么不需要覆盖模型文件。
        save_best_only=True, # 这就可以始终保存在训练过程中见到的最佳模型
    )
]

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc']) # 回调函数已经指定要监控精度，所以它应该是模型指标的一部分

model.fit(x,y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation=(x_val, y_val) # 由于回调函数要监控验证损失和验证精度，所以在调用fit时需要传入validation_data（验证数据）
         )

### ReduceLROnPlateau回调函数
如果验证损失不再改善，你可以使用这个回调函数来降低学习率。在训练过程中如果出现了损失平台
（loss plateau），那么增大或减小学习率都是跳出局部最小值的有效策略。

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
      monitor='val_loss', # 监控模型的验证损失
      factor=0.1, # 触发时将学习率除以10
      patience=10, # 如果验证损失在10轮内都没有改善，那么就触发这个回调函数
    )
]

model.fit(x,y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val) # 因为回调函数要监控验证损失，所以你需要在调用fit时传入validation_data（验证数据）
         )

### 编写自己的回调函数
如果你需要在训练过程中采取特定行动，而这项行动又没有包含在内置回调函数中，那么可以编写你自己的回调函数。

回调函数的实现方式是创建 keras.callbacks.Callback 类的子类。然后你可以实现下面这些方法（从名称中即可看出这些方法的作用），它们分别在训练过程中的不同时间点被调用。
- on_epoch_begin   ←------ 在每轮开始时被调用
- on_epoch_end   ←------ 在每轮结束时被调用
- on_batch_begin   ←------ 在处理每个批量之前被调用
- on_batch_end   ←------ 在处理每个批量之后被调用　
- on_train_begin   ←------ 在训练开始时被调用
- on_train_end   ←------ 在训练结束时被调用

这些方法被调用时都有一个 logs 参数，这个参数是一个字典，里面包含前一个批量、前一个轮次或前一次训练的信息，即训练指标和验证指标等。此外，回调函数还可以访问下列属性。
- self.model：调用回调函数的模型实例。
- self.validation_data：传入 fit 作为验证数据的值。

下面是一个自定义回调函数的简单示例，它可以在每轮结束后将模型每层的激活保存到硬盘（格式为 Numpy 数组），这个激活是对验证集的第一个样本计算得到的。

In [ ]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        self.model = model # 在训练之前由父模型调用，告诉回调函数是哪个模型在调用它
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input,
                                                   layer_outputs) # 模型实例，返回每层的激活
    
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation data')
        validation_sample = self.validation_data[0][0:1] # 获取验证数据的第一个输入样本
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()